# 3️⃣ PTL 변환: 학습된 모델 모바일 변환

이 노트북은 2_train.ipynb에서 학습한 3가지 KD 모델을 PTL 파일로 변환합니다.

**변환 대상:**
- Output KD (`weights/finetuned_output.pth`)
- Feature KD (`weights/finetuned_feature.pth`)
- FaKD (`weights/finetuned_fakd.pth`)

**입력 해상도:** 320, 640, 1280, 1920 (4가지)

**총 생성 파일:** 3 모델 × 4 해상도 = 12개 PTL 파일

**주의:** 메모리 사용량이 크므로 순차적으로 한 개씩 변환합니다.

## 3.0 환경 설정

In [ ]:
import os
import sys
import gc

# 상대 경로 설정
project_dir = os.path.dirname(os.path.abspath('__file__'))
if not project_dir or project_dir == '.':
    project_dir = os.getcwd()

# 가상환경 경로 설정
home_dir = os.path.expanduser('~')
venv_path = os.path.join(home_dir, 'my_ssl/ssl_env')
site_packages = os.path.join(venv_path, 'lib', 'python3.12', 'site-packages')
python_exec = os.path.join(venv_path, 'bin', 'python')

if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

%cd {project_dir}

import torch

print(f"✅ 환경 설정 완료")
print(f"   Project: {project_dir}")
print(f"   Python: {python_exec}")
print(f"   PyTorch: {torch.__version__}")
if torch.cuda.is_available():
    print(f"   CUDA: {torch.cuda.get_device_name(0)}")

## 3.1 변환 대상 모델 확인

In [ ]:
print("📁 변환 대상 모델 확인")
print("="*60)

models = [
    ('output', 'weights/finetuned_output.pth', 'Output KD'),
    ('feature', 'weights/finetuned_feature.pth', 'Feature KD'),
    ('fakd', 'weights/finetuned_fakd.pth', 'FaKD'),
]

available_models = []
for model_id, path, name in models:
    if os.path.exists(path):
        size = os.path.getsize(path) / (1024 * 1024)
        print(f"✅ {name:12} | {path} ({size:.1f} MB)")
        available_models.append((model_id, path, name))
    else:
        print(f"❌ {name:12} | {path} (없음)")

print(f"\n📊 변환 가능한 모델: {len(available_models)}/3")

if len(available_models) == 0:
    print("\n⚠️ 변환할 모델이 없습니다!")
    print("   먼저 2_train.ipynb를 실행하여 모델을 학습하세요.")

---
## 3.2 Output KD 모델 PTL 변환

In [ ]:
# Output KD PTL 변환 (320, 640, 1280, 1920)
print("="*60)
print("🔄 Output KD 모델 PTL 변환")
print("="*60)

weights_path = "weights/finetuned_output.pth"
input_sizes = [320, 640, 1280, 1920]

if not os.path.exists(weights_path):
    print(f"❌ 가중치 파일이 없습니다: {weights_path}")
else:
    for i, size in enumerate(input_sizes, 1):
        print(f"\n[{i}/4] Output KD - 입력 크기: {size}x{size}")
        print("-"*40)
        
        output_path = f"weights/output_kd_{size}.ptl"
        
        if os.path.exists(output_path):
            print(f"✅ 이미 존재: {output_path}")
            continue
        
        !{python_exec} scripts/convert_to_ptl.py \
            --weights {weights_path} \
            --input_size {size} \
            --upscale 2 \
            --output {output_path}
        
        # 메모리 정리 (중요!)
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        if os.path.exists(output_path):
            print(f"✅ 완료: {output_path}")
        else:
            print(f"❌ 실패: {output_path}")

print("\n" + "="*60)
print("✅ Output KD PTL 변환 완료!")

---
## 3.3 Feature KD 모델 PTL 변환

In [ ]:
# Feature KD PTL 변환 (320, 640, 1280, 1920)
print("="*60)
print("🔄 Feature KD 모델 PTL 변환")
print("="*60)

weights_path = "weights/finetuned_feature.pth"
input_sizes = [320, 640, 1280, 1920]

if not os.path.exists(weights_path):
    print(f"❌ 가중치 파일이 없습니다: {weights_path}")
else:
    for i, size in enumerate(input_sizes, 1):
        print(f"\n[{i}/4] Feature KD - 입력 크기: {size}x{size}")
        print("-"*40)
        
        output_path = f"weights/feature_kd_{size}.ptl"
        
        if os.path.exists(output_path):
            print(f"✅ 이미 존재: {output_path}")
            continue
        
        !{python_exec} scripts/convert_to_ptl.py \
            --weights {weights_path} \
            --input_size {size} \
            --upscale 2 \
            --output {output_path}
        
        # 메모리 정리 (중요!)
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        if os.path.exists(output_path):
            print(f"✅ 완료: {output_path}")
        else:
            print(f"❌ 실패: {output_path}")

print("\n" + "="*60)
print("✅ Feature KD PTL 변환 완료!")

---
## 3.4 FaKD 모델 PTL 변환

In [ ]:
# FaKD PTL 변환 (320, 640, 1280, 1920)
print("="*60)
print("🔄 FaKD 모델 PTL 변환")
print("="*60)

weights_path = "weights/finetuned_fakd.pth"
input_sizes = [320, 640, 1280, 1920]

if not os.path.exists(weights_path):
    print(f"❌ 가중치 파일이 없습니다: {weights_path}")
else:
    for i, size in enumerate(input_sizes, 1):
        print(f"\n[{i}/4] FaKD - 입력 크기: {size}x{size}")
        print("-"*40)
        
        output_path = f"weights/fakd_kd_{size}.ptl"
        
        if os.path.exists(output_path):
            print(f"✅ 이미 존재: {output_path}")
            continue
        
        !{python_exec} scripts/convert_to_ptl.py \
            --weights {weights_path} \
            --input_size {size} \
            --upscale 2 \
            --output {output_path}
        
        # 메모리 정리 (중요!)
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        if os.path.exists(output_path):
            print(f"✅ 완료: {output_path}")
        else:
            print(f"❌ 실패: {output_path}")

print("\n" + "="*60)
print("✅ FaKD PTL 변환 완료!")

---
## 3.5 생성된 PTL 파일 확인

In [ ]:
print("="*60)
print("📁 생성된 PTL 파일 목록")
print("="*60)

# 원본 모델 PTL
print("\n🔹 원본 CATANet-L x2:")
!ls -lh weights/CATANet-L_x2_*.ptl 2>/dev/null || echo "   없음"

# KD 모델 PTL
print("\n🔹 Output KD:")
!ls -lh weights/output_kd_*.ptl 2>/dev/null || echo "   없음"

print("\n🔹 Feature KD:")
!ls -lh weights/feature_kd_*.ptl 2>/dev/null || echo "   없음"

print("\n🔹 FaKD:")
!ls -lh weights/fakd_kd_*.ptl 2>/dev/null || echo "   없음"

# 총 개수
print("\n" + "="*60)
total_ptl = len([f for f in os.listdir('weights') if f.endswith('.ptl')]) if os.path.exists('weights') else 0
print(f"📊 총 PTL 파일: {total_ptl}개")
print("   - 원본: 4개 (320, 640, 1280, 1920)")
print("   - KD 모델: 12개 (3 모델 × 4 해상도)")
print("="*60)
print("\n💡 다음 단계: 4_benchmark.ipynb에서 성능 평가")